In [1]:
#! python -m pip install gekko

In [2]:
import numpy as np
import pandas as pd
from gekko import GEKKO

In [3]:
np.random.seed(1)
df_returns=pd.DataFrame(np.random.rand(100,4)-0.25,columns=list('ABCD'))
df_returns

,A,B,C,D
0,0.167022,0.470324,-0.249886,0.052333
1,-0.103244,-0.157661,-0.063740,0.095561
2,0.146767,0.288817,0.169195,0.435220
3,-0.045548,0.628117,-0.222612,0.420468
4,0.167305,0.308690,-0.109613,-0.051899
...,...,...,...,...
95,0.191904,0.331577,0.739752,-0.046094
96,-0.002267,0.012173,0.500172,0.206975
97,-0.193071,0.258516,-0.038040,0.548604
98,0.047331,-0.222394,0.343432,0.593840


In [4]:
weights=pd.Series([0.15,0.35,0.25,0.25],index=list('ABCD'))
weights

A    0.15
B    0.35
C    0.25
D    0.25
dtype: float64

In [5]:
0.167022*0.15,-0.103244*0.15

(0.0250533, -0.0154866)

In [6]:
df_returns.mean()[0]

0.24384782688578396

In [7]:
df_returns*weights

,A,B,C,D
0,0.025053,0.164614,-0.062471,0.013083
1,-0.015487,-0.055181,-0.015935,0.023890
2,0.022015,0.101086,0.042299,0.108805
3,-0.006832,0.219841,-0.055653,0.105117
4,0.025096,0.108041,-0.027403,-0.012975
...,...,...,...,...
95,0.028786,0.116052,0.184938,-0.011523
96,-0.000340,0.004261,0.125043,0.051744
97,-0.028961,0.090481,-0.009510,0.137151
98,0.007100,-0.077838,0.085858,0.148460


In [8]:
def returns_objective_function(x,df_returns):
    result=(df_returns*x).mean().sum()
    return result

In [9]:
x=np.ones(4)
x

array([1., 1., 1., 1.])

In [10]:
returns_objective_function(weights,df_returns)

0.2623920970760596

In [11]:
m=GEKKO()

In [12]:
x1=m.Var(lb=0.01,ub=0.8)
x2=m.Var(lb=0.01,ub=0.8)
x3=m.Var(lb=0.01,ub=0.8)
x4=m.Var(lb=0.01,ub=0.75)

In [13]:
m.Equation(x1+x2+x3+x4==1)#Objective Function
m.Equation(x1+x2==0.3)

In [14]:
df_returns.mean()[3]

0.29300382583510026

In [15]:
m.Maximize(0.24384782688578396*x1+0.2789166018980218*x2+0.21977262368043718*x3+0.29300382583510026*x4)#subject function

In [16]:
m.solve(disp=False)

In [17]:
print(x1.value[0],x2.value[0],x3.value[0],x4.value[0])
print(m.options.objfcnval)

0.01 0.29000000859 0.01 0.69000000998
-0.2876946642


In [18]:
from scipy.optimize import minimize

In [19]:
def returns_objective_function(x,df_returns):
    result=-(df_returns*x).mean().sum()
    return result

In [20]:
x=np.ones(4)
x

array([1., 1., 1., 1.])

In [21]:
returns_objective_function(weights,df_returns)

-0.2623920970760596

In [22]:
cons=(
    {'type':'eq','fun':lambda x: np.sum(x)-1},
    #{'type':'ieq','fun':lambda x: x[1]-0.24},
    {'type':'ineq','fun':lambda x: x[0]+x[1]-0.30}
)
b=(0.01,0.8)
bnds=(b,b,b,(0.01,0.75))
bnds

((0.01, 0.8), (0.01, 0.8), (0.01, 0.8), (0.01, 0.75))

In [23]:
result=minimize(returns_objective_function,weights,(df_returns),bounds=bnds,constraints=cons,method='SLSQP')

In [24]:
result

     fun: -0.28769465888230783
     jac: array([-0.24384783, -0.2789166 , -0.21977263, -0.29300383])
 message: 'Optimization terminated successfully'
    nfev: 35
     nit: 7
    njev: 7
  status: 0
 success: True
       x: array([0.01, 0.29, 0.01, 0.69])

In [25]:
(result['x']*df_returns).mean().sum()

0.28769465888230783